In [1]:
using LinearAlgebra
using Distributions
using Optim
using Random
using StatsFuns
using JuMP
using MosekTools
using JLD2
include("ETO.jl")
include("RO.jl")
include("Data.jl")
include("Estimate.jl")
include("Performance.jl")

greedy_search_optimal_price (generic function with 1 method)

In [2]:
current_dir = pwd()
parent_dir = dirname(current_dir)
grand_pa_dir = dirname(parent_dir)
data_dir = string(grand_pa_dir, "/Data/")
if !isdir(data_dir)
    mkpath(data_dir)
end

## Functions

In [3]:
function generate_Input_Data(S_train,S_test,iterations, N, N_u, K, offdiag_sign,max_offdiag,P_bar)
    Input_Data = Dict()
    for iter in 1:iterations
        A_true, B_true = Generate_Coef(N_u, N,max_offdiag,offdiag_sign);
        U_train, P_train = Generate_Feat_Data(N_u, N, S_train);
        U_test, P_test = Generate_Feat_Data(N_u, N, S_test);

        Input_Data["iter=$(iter)_Obs_Feat"] = U_test[1,:];
        Input_Data["iter=$(iter)_A_true"] = A_true;
        Input_Data["iter=$(iter)_B_true"] = B_true;
        Input_Data["iter=$(iter)_P_dag"] = round.(rand(N, K) .* P_bar; digits=2);
        Input_Data["iter=$(iter)_U_train"] = U_train;
        Input_Data["iter=$(iter)_P_train"] = P_train;

        A_hat, B_hat = Estimate_MNL_Para(U_train, P_train, S_train, N, N_u, N, A_true, B_true);

        Input_Data["iter=$(iter)_A_hat"] = A_hat
        Input_Data["iter=$(iter)_B_hat"] = B_hat
    end
    return Input_Data
end

generate_Input_Data (generic function with 1 method)

In [4]:
function Run_Oracle(iterations, N, N_u, K, Input_Data)
    RST_ = Dict()
    for iter in 1:iterations
        Obs_Feat = Input_Data["iter=$(iter)_Obs_Feat"]
        A_true = Input_Data["iter=$(iter)_A_true"]
        B_true = Input_Data["iter=$(iter)_B_true"]
        P_dag = Input_Data["iter=$(iter)_P_dag"]
        
        obj_ETO,X_ETO,time_ETO = Solve_ETO(N,N_u,K,A_true,B_true,Obs_Feat,P_dag)
        rev_ETO, price_ETO = compute_oof(X_ETO, A_true, B_true, Obs_Feat, P_dag)
        # println("rev_ETO=",round(rev_ETO,digits=6),",price_ETO = ",price_ETO)
        RST_["iter=$(iter)_Rev"] = rev_ETO
        RST_["iter=$(iter)_Price"] = price_ETO
        RST_["iter=$(iter)_Obj"] = obj_ETO
    end
    return RST_
end

Run_Oracle (generic function with 1 method)

In [5]:
function Run_ETO(iterations, N, N_u, K, Input_Data)
    RST_ETO = Dict()
    for iter in 1:iterations
        Obs_Feat = Input_Data["iter=$(iter)_Obs_Feat"]
        A_hat = Input_Data["iter=$(iter)_A_hat"]
        B_hat = Input_Data["iter=$(iter)_B_hat"]
        A_true = Input_Data["iter=$(iter)_A_true"]
        B_true = Input_Data["iter=$(iter)_B_true"]
        P_dag = Input_Data["iter=$(iter)_P_dag"]
        
        obj_ETO,X_ETO,time_ETO = Solve_ETO(N,N_u,K,A_hat,B_hat,Obs_Feat,P_dag)
        rev_ETO, price_ETO = compute_oof(X_ETO, A_true, B_true, Obs_Feat, P_dag)
        # println("rev_ETO=",round(rev_ETO,digits=6),",price_ETO = ",price_ETO)
        RST_ETO["iter=$(iter)_Rev"] = rev_ETO
        RST_ETO["iter=$(iter)_Price"] = price_ETO
        RST_ETO["iter=$(iter)_Obj"] = obj_ETO
    end
    return RST_ETO
end

Run_ETO (generic function with 1 method)

In [6]:
function search_opt_price(N,p_lb,p_ub,b_n)
    model = Model(Mosek.Optimizer)
    set_attribute(model, "QUIET", true)
    # 定义变量
    @variable(model, price[1:N])                      # y_{nk}
    @constraint(model, price .>= p_lb)
    @constraint(model, price .<= p_ub)
    @objective(model, Max,b_n' * price )
    optimize!(model)
    obj_val = objective_value(model)
    return obj_val
end

search_opt_price (generic function with 1 method)

In [7]:
function Calculate_Hyper_Param(bd_coef, iterations, N, N_u, K, Input_Data)
    for iter in 1:iterations
        Obs_Feat = Input_Data["iter=$(iter)_Obs_Feat"]
        A_true = Input_Data["iter=$(iter)_A_true"]
        B_true = Input_Data["iter=$(iter)_B_true"]
        P_dag = Input_Data["iter=$(iter)_P_dag"]
        A_hat = Input_Data["iter=$(iter)_A_hat"]
        B_hat = Input_Data["iter=$(iter)_B_hat"]

        A_lb = A_hat .- bd_coef .* abs.(ones(N,N_u));
        A_ub = A_hat .+ bd_coef .* abs.(ones(N,N_u));
        B_lb = B_hat .- bd_coef .* abs.(ones(N,N));
        B_ub = B_hat .+ bd_coef .* abs.(ones(N,N));
        
        p_ub = vec(maximum(P_dag,dims=2))
        p_lb = vec(minimum(P_dag,dims=2))
        p_max = maximum(p_ub)
        p_min = minimum(p_lb)
        Obs_Feat_Trun = [max(-Obs_Feat[ind],0) for ind in 1:N_u]
        psi_lb = zeros(N)
        for n in 1:N
            b_n = B_lb[n,:]
            obj_n = search_opt_price(N,p_lb,p_ub,b_n)
            psi_lb[n] = -exp(-Obs_Feat_Trun' * (A_ub[n,:] - A_lb[n,:]) + Obs_Feat' * A_lb[n,:] + obj_n)*(p_max-p_min)
        end
        
        Input_Data["iter=$(iter)_psi_lb"] = psi_lb
        Input_Data["iter=$(iter)_psi_ub"] = zeros(N)
        Input_Data["iter=$(iter)_phi_lb"] = [p_min - p_max for i in 1:N]
        Input_Data["iter=$(iter)_phi_ub"] = zeros(N)

        Input_Data["iter=$(iter)_A_lb"] = A_lb
        Input_Data["iter=$(iter)_A_ub"] = A_ub
        Input_Data["iter=$(iter)_B_lb"] = B_lb
        Input_Data["iter=$(iter)_B_ub"] = B_ub
    end
    return Input_Data
end

Calculate_Hyper_Param (generic function with 1 method)

In [8]:
function Run_RO(bd_coef, iterations, N, N_u, K, Input_Data,psi_coef,model_name)
    RST_RO = Dict(); RST_RO_Two_Side = Dict()
    for iter in 1:iterations
        Obs_Feat = Input_Data["iter=$(iter)_Obs_Feat"]
        A_true = Input_Data["iter=$(iter)_A_true"]
        B_true = Input_Data["iter=$(iter)_B_true"]
        P_dag = Input_Data["iter=$(iter)_P_dag"]

        psi_lb = Input_Data["iter=$(iter)_psi_lb"]
        psi_ub = Input_Data["iter=$(iter)_psi_ub"]
        phi_lb = Input_Data["iter=$(iter)_phi_lb"]
        phi_ub = Input_Data["iter=$(iter)_phi_ub"]

        A_lb = Input_Data["iter=$(iter)_A_lb"]
        A_ub = Input_Data["iter=$(iter)_A_ub"]
        B_lb = Input_Data["iter=$(iter)_B_lb"]
        B_ub = Input_Data["iter=$(iter)_B_ub"]
        if model_name == "Two_Side"
            obj_RO,X_RO,time_RO = Solve_RO(N,N_u,K,A_lb,A_ub,B_lb,B_ub,Obs_Feat,P_dag,psi_lb,psi_ub,phi_lb,phi_ub)
        else
            obj_RO,X_RO,time_RO = Solve_RO_one_side_exp(N,N_u,K,A_lb,A_ub,B_lb,B_ub,Obs_Feat,P_dag,psi_lb,psi_ub)
        end
        
        rev_RO, price_RO = compute_oof(X_RO, A_true, B_true, Obs_Feat, P_dag)
        println("rev_RO = ",round(rev_RO,digits=6),",price_RO = ",price_RO)
        RST_RO["iter=$(iter)_Rev"] = rev_RO
        RST_RO["iter=$(iter)_Price"] = price_RO
        RST_RO["iter=$(iter)_Obj"] = obj_RO
    end
    return RST_RO
end

Run_RO (generic function with 1 method)

In [15]:
function Run_RO_with_Given_bd_coef(bd_coef, iterations, N, N_u, K, Input_Data, psi_coef,RST_ETO,model_name)
    RST_RO = Run_RO(bd_coef, iterations, N, N_u, K, Input_Data,psi_coef,model_name);
    Rev_ETO = [RST_ETO["iter=$(iter)_Rev"] for iter in 1:iterations]
    Rev_RO = [RST_RO["iter=$(iter)_Rev"] for iter in 1:iterations]
    # println("Rev_ETO = ",round.(Rev_ETO,digits=4))
    println("Rev_RO/Rev_ETO = ",round.(Rev_RO./Rev_ETO,digits=4), ", Average = ",round.(mean(Rev_RO./Rev_ETO),digits=4))
    return RST_RO
end

Run_RO_with_Given_bd_coef (generic function with 2 methods)

## Evaluate

### offdiag is mix

In [10]:
Random.seed!(1)
N_u = 1
N = 5
S_train = 1000
S_test = 1
K = 10   # 每个产品的选择项数量
max_offdiag = 0.3
offdiag_sign = "mix"  # "positive" or "negative"
P_bar = 1.0
iterations = 10

10

- Generate Data

In [11]:
Input_Data = generate_Input_Data(S_train,S_test,iterations, N, N_u, K, offdiag_sign,max_offdiag,P_bar);
bd_coef = 0.0
Input_Data = Calculate_Hyper_Param(bd_coef, iterations, N, N_u, K, Input_Data);
save(string(data_dir, "Input_Data_",offdiag_sign,".jld2"), Input_Data);

- Run Oracle

In [12]:
RST_Oracle = Run_Oracle(iterations, N, N_u, K, Input_Data);

- Run ETO

In [13]:
RST_ETO = Run_ETO(iterations, N, N_u, K, Input_Data);
Rev_ETO = [RST_ETO["iter=$(iter)_Rev"] for iter in 1:iterations]
Rev_Oracle = [RST_Oracle["iter=$(iter)_Rev"] for iter in 1:iterations]
println("Rev_ETO/Rev_Oracle = ",round.(Rev_ETO./Rev_Oracle,digits=4))

Rev_ETO/Rev_Oracle = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9808, 1.0]


- Run RO

In [17]:
model_name="Two_Side"
RST_RO = Run_RO_with_Given_bd_coef(bd_coef, iterations, N, N_u, K, Input_Data, psi_coef,RST_ETO,model_name);
Rev_RO = [RST_RO["iter=$(iter)_Rev"] for iter in 1:iterations]
println("Rev_RO/Rev_ETO = ",round.(Rev_RO./Rev_ETO,digits=4))

rev_RO = 0.733612,price_RO = [0.91, 0.96, 0.87, 0.98, 0.74]
rev_RO = 0.645775,price_RO = [0.92, 0.9, 0.74, 0.97, 0.69]
rev_RO = 0.764505,price_RO = [0.97, 0.94, 0.96, 0.91, 0.94]
rev_RO = 0.743258,price_RO = [0.89, 0.81, 0.88, 0.99, 0.87]
rev_RO = 0.683563,price_RO = [0.98, 0.92, 0.96, 1.0, 0.88]
rev_RO = 0.60551,price_RO = [0.97, 0.76, 0.88, 0.9, 0.98]
rev_RO = 0.666927,price_RO = [0.96, 0.83, 0.93, 0.91, 0.97]
rev_RO = 0.74936,price_RO = [0.95, 0.85, 0.91, 0.95, 0.95]
rev_RO = 0.6788,price_RO = [0.99, 0.98, 0.88, 0.95, 0.72]
rev_RO = 0.682955,price_RO = [0.88, 0.92, 0.84, 0.82, 0.91]
Rev_RO/Rev_ETO = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], Average = 1.0
Rev_RO/Rev_ETO = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


### offdiag is zero

In [18]:
offdiag_sign = "zero" 
Input_Data = generate_Input_Data(S_train,S_test,iterations, N, N_u, K, offdiag_sign,max_offdiag,P_bar);
bd_coef = 0.0
Input_Data = Calculate_Hyper_Param(bd_coef, iterations, N, N_u, K, Input_Data);
save(string(data_dir, "Input_Data_",offdiag_sign,".jld2"), Input_Data);

In [19]:
RST_Oracle = Run_Oracle(iterations, N, N_u, K, Input_Data);

In [20]:
RST_ETO = Run_ETO(iterations, N, N_u, K, Input_Data);
Rev_ETO = [RST_ETO["iter=$(iter)_Rev"] for iter in 1:iterations]
Rev_Oracle = [RST_Oracle["iter=$(iter)_Rev"] for iter in 1:iterations]
println("Rev_ETO/Rev_Oracle = ",round.(Rev_ETO./Rev_Oracle,digits=4))

Rev_ETO/Rev_Oracle = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


In [21]:
model_name="Two_Side"
Run_RO_with_Given_bd_coef(bd_coef, iterations, N, N_u, K, Input_Data, psi_coef,RST_ETO,model_name);

rev_RO = 0.688269,price_RO = [0.85, 0.89, 0.93, 0.94, 0.97]
rev_RO = 0.690565,price_RO = [0.9, 0.97, 0.9, 0.96, 0.99]
rev_RO = 0.665666,price_RO = [0.87, 0.71, 0.85, 0.98, 0.95]
rev_RO = 0.698709,price_RO = [0.89, 0.78, 0.98, 0.94, 0.8]
rev_RO = 0.747399,price_RO = [0.95, 0.98, 0.93, 0.88, 0.97]
rev_RO = 0.657518,price_RO = [0.77, 0.83, 0.93, 0.9, 0.88]
rev_RO = 0.66975,price_RO = [0.93, 0.79, 0.76, 0.95, 0.99]
rev_RO = 0.723362,price_RO = [0.98, 0.97, 0.76, 0.87, 0.95]
rev_RO = 0.765142,price_RO = [0.95, 0.97, 0.97, 0.96, 0.92]
rev_RO = 0.72611,price_RO = [0.84, 0.76, 0.99, 0.94, 0.95]
Rev_RO/Rev_ETO = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], Average = 1.0


### offdiag is positive

In [22]:
offdiag_sign = "positive" 
Input_Data = generate_Input_Data(S_train,S_test,iterations, N, N_u, K, offdiag_sign,max_offdiag,P_bar);
bd_coef = 0.0
Input_Data = Calculate_Hyper_Param(bd_coef, iterations, N, N_u, K, Input_Data)
save(string(data_dir, "Input_Data_",offdiag_sign,".jld2"), Input_Data)

In [23]:
RST_Oracle = Run_Oracle(iterations, N, N_u, K, Input_Data);

In [24]:
RST_ETO = Run_ETO(iterations, N, N_u, K, Input_Data);
Rev_ETO = [RST_ETO["iter=$(iter)_Rev"] for iter in 1:iterations]
Rev_Oracle = [RST_Oracle["iter=$(iter)_Rev"] for iter in 1:iterations]
println("Rev_ETO/Rev_Oracle = ",round.(Rev_ETO./Rev_Oracle,digits=4))

Rev_ETO/Rev_Oracle = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


In [25]:
model_name="Two_Side"
RST_RO = Run_RO_with_Given_bd_coef(bd_coef, iterations, N, N_u, K, Input_Data, psi_coef,RST_ETO,model_name);

rev_RO = 0.790502,price_RO = [0.93, 0.96, 0.79, 0.93, 0.96]
rev_RO = 0.697667,price_RO = [0.85, 0.85, 0.92, 0.75, 0.89]
rev_RO = 0.793466,price_RO = [0.94, 0.88, 0.95, 0.82, 0.94]
rev_RO = 0.779759,price_RO = [0.96, 0.99, 0.91, 0.8, 1.0]
rev_RO = 0.709217,price_RO = [0.98, 0.66, 0.88, 0.99, 0.78]
rev_RO = 0.786794,price_RO = [0.92, 0.95, 0.98, 0.92, 0.96]
rev_RO = 0.766982,price_RO = [1.0, 0.88, 0.95, 0.76, 0.91]
rev_RO = 0.742852,price_RO = [0.89, 0.96, 0.9, 0.91, 0.79]
rev_RO = 0.835651,price_RO = [0.94, 0.95, 0.93, 0.95, 0.98]
rev_RO = 0.794992,price_RO = [0.98, 0.84, 0.98, 0.95, 0.98]
Rev_RO/Rev_ETO = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], Average = 1.0


### offdiag is negative

In [26]:
offdiag_sign = "negative" 
Input_Data = generate_Input_Data(S_train,S_test,iterations, N, N_u, K, offdiag_sign,max_offdiag,P_bar);
bd_coef = 0.0
Input_Data = Calculate_Hyper_Param(bd_coef, iterations, N, N_u, K, Input_Data)
save(string(data_dir, "Input_Data_",offdiag_sign,".jld2"), Input_Data)

In [27]:
RST_Oracle = Run_Oracle(iterations, N, N_u, K, Input_Data);

In [28]:
RST_ETO = Run_ETO(iterations, N, N_u, K, Input_Data);
Rev_ETO = [RST_ETO["iter=$(iter)_Rev"] for iter in 1:iterations]
Rev_Oracle = [RST_Oracle["iter=$(iter)_Rev"] for iter in 1:iterations]
println("Rev_ETO/Rev_Oracle = ",round.(Rev_ETO./Rev_Oracle,digits=4))

Rev_ETO/Rev_Oracle = [0.9961, 1.0, 0.9843, 1.0, 0.9879, 1.0, 1.0, 1.0, 1.0, 1.0]


In [29]:
model_name="Two_Side"
RST_RO = Run_RO_with_Given_bd_coef(bd_coef, iterations, N, N_u, K, Input_Data, psi_coef,RST_ETO,model_name);

rev_RO = 0.572319,price_RO = [0.91, 0.8, 0.96, 0.79, 0.98]
rev_RO = 0.65257,price_RO = [0.84, 0.95, 0.96, 0.92, 0.86]
rev_RO = 0.58621,price_RO = [0.96, 0.8, 0.83, 0.65, 0.93]
rev_RO = 0.700351,price_RO = [0.98, 0.94, 0.86, 0.88, 0.87]
rev_RO = 0.591016,price_RO = [1.0, 0.98, 0.82, 0.99, 0.78]
rev_RO = 0.619304,price_RO = [0.98, 0.96, 0.95, 0.54, 0.99]
rev_RO = 0.618306,price_RO = [0.89, 0.82, 0.91, 0.95, 0.95]
rev_RO = 0.660026,price_RO = [0.89, 0.89, 0.99, 1.0, 0.83]
rev_RO = 0.597654,price_RO = [0.77, 0.93, 0.95, 0.99, 0.77]
rev_RO = 0.631661,price_RO = [0.94, 0.98, 0.93, 0.87, 0.75]
Rev_RO/Rev_ETO = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], Average = 1.0


## Check Correctness, max_offdiag = 1.0

In [31]:
N_u = 1
N = 5
S_train = 1000
S_test = 1
K = 10   # 每个产品的选择项数量
P_bar = 1.0
iterations = 10

10

In [32]:
Random.seed!(1)
max_offdiag = 1.0
offdiag_sign = "mix"  # "positive" or "negative"

"mix"

In [33]:
Input_Data = generate_Input_Data(S_train,S_test,iterations, N, N_u, K, offdiag_sign,max_offdiag,P_bar);
bd_coef = 0.0;
Input_Data = Calculate_Hyper_Param(bd_coef, iterations, N, N_u, K, Input_Data);
# save(string(data_dir, "Input_Data_",offdiag_sign,"_max_offdiag=1.jld2"), Input_Data)

Dict{Any, Any} with 160 entries:
  "iter=7_B_true"   => [-2.80069 0.251263 … -0.951699 0.363152; 0.654146 -3.373…
  "iter=5_B_hat"    => [-3.10939 0.642363 … -0.478885 -0.645334; -0.416721 -2.0…
  "iter=7_A_ub"     => [0.846102; 1.11471; … ; 0.372782; 1.21583;;]
  "iter=5_psi_ub"   => [0.0, 0.0, 0.0, 0.0, 0.0]
  "iter=8_phi_ub"   => [0.0, 0.0, 0.0, 0.0, 0.0]
  "iter=7_B_ub"     => [-2.68912 0.911353 … -1.13312 -0.23372; 0.590211 -3.9525…
  "iter=7_B_lb"     => [-2.68912 0.911353 … -1.13312 -0.23372; 0.590211 -3.9525…
  "iter=5_A_lb"     => [0.921217; 0.256813; … ; 0.379923; -0.128712;;]
  "iter=4_A_hat"    => [0.192624; 1.23652; … ; 1.15842; 1.02723;;]
  "iter=4_phi_lb"   => [-0.91, -0.91, -0.91, -0.91, -0.91]
  "iter=3_A_true"   => [0.418871; 0.345158; … ; 0.6974; 0.913239;;]
  "iter=8_B_hat"    => [-1.37777 0.466814 … 0.386345 -0.619382; -0.129072 -2.57…
  "iter=8_P_train"  => [0.44 0.75 … 0.09 0.31; 0.19 0.24 … 0.15 0.28; … ; 0.48 …
  "iter=5_A_hat"    => [0.921217; 0.256813; … ; 0.

In [34]:
RST_Oracle = Run_Oracle(iterations, N, N_u, K, Input_Data);

In [35]:
RST_ETO = Run_ETO(iterations, N, N_u, K, Input_Data);

In [36]:
model_name="Two_Side"
RST_RO = Run_RO_with_Given_bd_coef(bd_coef, iterations, N, N_u, K, Input_Data, 0.01,RST_ETO,model_name);

rev_RO = 0.592124,price_RO = [0.91, 0.96, 0.87, 0.98, 0.74]
rev_RO = 0.338799,price_RO = [0.92, 0.71, 0.69, 0.52, 0.69]
rev_RO = 0.628102,price_RO = [0.97, 0.94, 0.96, 0.91, 0.94]
rev_RO = 0.601745,price_RO = [0.64, 0.81, 0.88, 0.99, 0.87]
rev_RO = 0.435418,price_RO = [0.41, 0.92, 0.96, 1.0, 0.78]
rev_RO = 0.268467,price_RO = [0.97, 0.76, 0.48, 0.9, 0.51]
rev_RO = 0.314258,price_RO = [0.66, 0.83, 0.93, 0.27, 0.57]
rev_RO = 0.641213,price_RO = [0.95, 0.85, 0.91, 0.95, 0.95]
rev_RO = 0.377471,price_RO = [0.76, 0.33, 0.88, 0.95, 0.85]
rev_RO = 0.570106,price_RO = [0.88, 0.92, 0.84, 0.82, 0.91]
Rev_RO/Rev_ETO = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], Average = 1.0


In [37]:
Rev_ETO = [RST_ETO["iter=$(iter)_Rev"] for iter in 1:iterations]
Rev_RO = [RST_RO["iter=$(iter)_Rev"] for iter in 1:iterations]
Rev_Oracle = [RST_Oracle["iter=$(iter)_Rev"] for iter in 1:iterations]
println("Rev_ETO/Rev_Oracle = ",round.(Rev_ETO./Rev_Oracle,digits=4))
println("Rev_RO/Rev_ETO = ",round.(Rev_RO./Rev_ETO,digits=4))

Rev_ETO/Rev_Oracle = [1.0, 0.9839, 1.0, 0.9896, 0.9966, 0.9677, 0.9177, 1.0, 0.986, 1.0]
Rev_RO/Rev_ETO = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


In [38]:
Price_ETO = [RST_ETO["iter=$(iter)_Price"] for iter in 1:iterations];
Price_RO = [RST_RO["iter=$(iter)_Price"] for iter in 1:iterations];

In [39]:
Price_ETO

10-element Vector{Vector{Float64}}:
 [0.91, 0.96, 0.87, 0.98, 0.74]
 [0.92, 0.71, 0.69, 0.52, 0.69]
 [0.97, 0.94, 0.96, 0.91, 0.94]
 [0.64, 0.81, 0.88, 0.99, 0.87]
 [0.41, 0.92, 0.96, 1.0, 0.78]
 [0.97, 0.76, 0.48, 0.9, 0.51]
 [0.66, 0.83, 0.93, 0.27, 0.57]
 [0.95, 0.85, 0.91, 0.95, 0.95]
 [0.76, 0.33, 0.88, 0.95, 0.85]
 [0.88, 0.92, 0.84, 0.82, 0.91]

In [40]:
Price_RO

10-element Vector{Vector{Float64}}:
 [0.91, 0.96, 0.87, 0.98, 0.74]
 [0.92, 0.71, 0.69, 0.52, 0.69]
 [0.97, 0.94, 0.96, 0.91, 0.94]
 [0.64, 0.81, 0.88, 0.99, 0.87]
 [0.41, 0.92, 0.96, 1.0, 0.78]
 [0.97, 0.76, 0.48, 0.9, 0.51]
 [0.66, 0.83, 0.93, 0.27, 0.57]
 [0.95, 0.85, 0.91, 0.95, 0.95]
 [0.76, 0.33, 0.88, 0.95, 0.85]
 [0.88, 0.92, 0.84, 0.82, 0.91]

In [ ]:
Obj_RO = [RST_RO["iter=$(iter)_Obj"] for iter in 1:iterations];
Obj_ETO = [RST_ETO["iter=$(iter)_Obj"] for iter in 1:iterations];

## S_train = 100

In [ ]:
Random.seed!(1)
N_u = 1
N = 5
S_train = 100
S_test = 1
K = 10   # 每个产品的选择项数量
max_offdiag = 0.3
offdiag_sign = "mix"  # "positive" or "negative"
P_bar = 1.0
iterations = 10

In [ ]:
Input_Data = generate_Input_Data(S_train,S_test,iterations, N, N_u, K, offdiag_sign,max_offdiag,P_bar);
bd_coef = 0.0
Input_Data = Calculate_Hyper_Param(bd_coef, iterations, N, N_u, K, Input_Data);
save(string(data_dir, "Input_Data_",offdiag_sign,".jld2"), Input_Data);

In [ ]:
RST_Oracle = Run_Oracle(iterations, N, N_u, K, Input_Data);

In [ ]:
RST_ETO = Run_ETO(iterations, N, N_u, K, Input_Data);

In [ ]:
model_name="Two_Side"
RST_RO = Run_RO_with_Given_bd_coef(bd_coef, iterations, N, N_u, K, Input_Data, 0.01,RST_ETO,model_name);

# ********* Below is the dubug history **************

#### iter = 5, ETO和RO的结果不一致

In [ ]:
Obj_ETO

In [ ]:
iter = 4
Obs_Feat = Input_Data["iter=$(iter)_Obs_Feat"]
A_true = Input_Data["iter=$(iter)_A_true"]
B_true = Input_Data["iter=$(iter)_B_true"]
A_hat = Input_Data["iter=$(iter)_A_hat"]
B_hat = Input_Data["iter=$(iter)_B_hat"]
P_dag = Input_Data["iter=$(iter)_P_dag"]

psi_lb = Input_Data["iter=$(iter)_psi_lb"]
psi_ub = Input_Data["iter=$(iter)_psi_ub"]

phi_lb = Input_Data["iter=$(iter)_phi_lb"]
phi_ub = Input_Data["iter=$(iter)_phi_ub"]

A_lb = Input_Data["iter=$(iter)_A_lb"] 
A_ub = Input_Data["iter=$(iter)_A_ub"]
B_lb = Input_Data["iter=$(iter)_B_lb"] 
B_ub = Input_Data["iter=$(iter)_B_ub"] 

In [ ]:
phi_ub

In [ ]:
obj_ETO,X_ETO,time_ETO = Solve_ETO(N,N_u,K,A_hat,B_hat,Obs_Feat,P_dag)

In [ ]:
price_ETO = sum(X_ETO .* P_dag,dims=2)

In [ ]:
obj_RO,X_RO,time_RO = Solve_RO_one_side_exp(N,N_u,K,A_lb,A_ub,B_lb,B_ub,Obs_Feat,P_dag,psi_lb,psi_ub)

In [ ]:
price_RO1 = sum(X_RO .* P_dag,dims=2)

#### 固定ETO模型中的solution为RO的solution,i.e., X_ETO = X_RO, 检查下结果是否和RO的结果一致

In [ ]:
u = Obs_Feat
p_dag = P_dag
X_given = X_RO
# X_given
model = Model(Mosek.Optimizer)
# set_attribute(model, "mioTolRelGap", 0.00000001)
set_attribute(model, "MSK_DPAR_MIO_TOL_REL_GAP", 1e-7)
set_attribute(model, "MSK_DPAR_MIO_TOL_ABS_GAP", 1e-7)
# 定义变量
@variable(model, delta)                           # 标量 δ
@variable(model, omega_lb[1:N,1:N_u] >= 0)            
@variable(model, omega_ub[1:N,1:N_u] >= 0)           
@variable(model, pi_lb[1:N,1:N] >= 0)     
@variable(model, pi_ub[1:N,1:N] >= 0)     

# exponential variables
@variable(model, psi_1[1:N])                   
@variable(model, psi_2[1:N])                   
@variable(model, psi_3[1:N])        

@variable(model, X[1:N, 1:K], Bin)        # 二进制变量 x_{jk}
@variable(model, Y[1:N,1:N,1:K] <= 0)    

for n in 1:N
    @constraint(model, omega_lb[n,:] .- omega_ub[n,:] .+ psi_3[n] .* u .== 0)
end

for n in 1:N
    @constraint(model, pi_lb[n,:] .- pi_ub[n,:] .+ sum(Y[n,:,:] .* p_dag,dims=2) .== 0)
end

@constraint(model, sum(omega_ub .* A_ub) - sum(omega_lb .* A_lb) + sum(pi_ub .* B_ub) - sum(pi_lb .* B_lb) + delta + sum(psi_2) <= 0)

for n in 1:N
    @constraint(model, delta + psi_1[n] - X[n,:]' * p_dag[n,:] <= 0)
end

for n in 1:N
    @constraint(model, [psi_3[n], psi_2[n], psi_1[n]] in MOI.DualExponentialCone())
end

for n in 1:N
    for j in 1:N
        for k in 1:K
            @constraint(model, Y[n,j,k] >= psi_lb[n] * X[j,k])
            @constraint(model, Y[n,j,k] <= psi_ub[n])
        end
    end
end    
for n in 1:N
    for j in 1:N
        @constraint(model, sum(Y[n,j,:]) == psi_3[n])
    end
end

@constraint(model, sum(X,dims=2) .== 1)

@constraint(model, X .- X_given .== 0)  # 固定X的值
@objective(model, Max, delta)

optimize!(model)
status = JuMP.termination_status(model)
# println("status: ", status)
# solution_summary(model)
if status == MOI.OPTIMAL
    obj_val = objective_value(model)
    X_val = round.(value.(X))
    solve_time = JuMP.solve_time(model)
else
    obj_val = NaN
    X_val = ones(N,N) .* NaN
    solve_time = NaN
end

In [ ]:
println("Objective value when X_ETO = X_RO:",obj_val)

#### 固定RO的solution为ETO的solution，检查RO的目标函数值是否等于ETO的目标函数值

- 计算模型中用到的L 和 M

In [ ]:
price = vec(sum(X_ETO .* P_dag,dims=2)) # 设定价格为ETO的价格
Obs_Feat_Trun = [max(-Obs_Feat[ind],0) for ind in 1:N_u]
Obs_Feat_Trun2 = [max(Obs_Feat[ind],0) for ind in 1:N_u]
psi_lb = zeros(N)
L_arr = zeros(N)
M_arr = zeros(N)
for n in 1:N
    L_arr[n] = -Obs_Feat_Trun' * (A_ub[n,:] - A_lb[n,:]) - Obs_Feat' * A_lb[n,:] - price' * B_lb[n,:]
    M_arr[n] = Obs_Feat_Trun2' * (A_ub[n,:] - A_lb[n,:]) + Obs_Feat' * A_lb[n,:] + price' * B_lb[n,:]
end

- <font color="red">我们首先考虑如下的简化模型：
\begin{equation}
    \begin{array}{rcll}
         & \sup & \displaystyle \delta \\
         & {\rm s.t.} 
         & \sum\limits_{n \in [N]}-\psi_n^3 L_n + \psi_{n}^{2}  \leq -\delta, \\
         &&  - \psi_n^3 \exp\left(\frac{\psi_n^2}{\psi_n^3}-1\right) \le p_{n} - \delta,& \forall n \in [N] \\
    \end{array}
\end{equation}
- 本质上来说，我们只考虑了如下的Exponential cone：
$\rho_n \geq \rho_0 \exp(a_u^\top u + b_u^\top p)$
- 结果是采用ETO的价格，RO模型得到的目标函数值不等于ETO模型的目标函数值
</font>

In [ ]:
model = Model(Mosek.Optimizer)
# set_attribute(model, "mioTolRelGap", 0.00000001)
set_attribute(model, "MSK_DPAR_MIO_TOL_REL_GAP", 1e-7)
set_attribute(model, "MSK_DPAR_MIO_TOL_ABS_GAP", 1e-7)
# 定义变量
@variable(model, delta)                           # 标量 δ
# exponential variables
@variable(model, psi_1[1:N])                   
@variable(model, psi_2[1:N])                   
@variable(model, psi_3[1:N])        

@constraint(model, -L_arr' * psi_3 + sum(psi_2) <= -delta)


for n in 1:N
    @constraint(model, psi_1[n] <= price[n] - delta)
end

for n in 1:N
    @constraint(model, [psi_3[n], psi_2[n], psi_1[n]] in MOI.DualExponentialCone())
end

@objective(model, Max, delta)

optimize!(model)
obj_val = objective_value(model);

In [ ]:
println("Objective value when X_RO = X_ETO:",obj_val)

- 我们之后考虑如下的简化模型：
\begin{equation}
    \begin{array}{rcll}
         & \sup & \displaystyle \delta \\
         & {\rm s.t.} 
         & \sum\limits_{n \in [N]}-\psi_n^3 L_n + \psi_{n}^{2}  - \phi_n^3 \exp\left(\frac{\phi_n^2}{\phi_n^3}-1\right) \leq -\delta, \\
         && - \phi_{n}^{3} M_n  - \psi_n^3 \exp\left(\frac{\psi_n^2}{\psi_n^3}-1\right) + \phi_{n}^{2} \le p_{n} - \delta,& \forall n \in [N] \\
    \end{array}
\end{equation}
<font color="red">
- 该模型考虑了如下两个的Exponential cone：
$\rho_n \geq \rho_0 \exp(a_u^\top u + b_u^\top p)$ 和 $\rho_n \leq \rho_0 \exp(a_u^\top u + b_u^\top p)$
- 结果是采用ETO的价格，RO模型得到的目标函数值等于ETO模型的目标函数值
</font>

In [ ]:
model = Model(Mosek.Optimizer)
# set_attribute(model, "mioTolRelGap", 0.00000001)
set_attribute(model, "MSK_DPAR_MIO_TOL_REL_GAP", 1e-7)
set_attribute(model, "MSK_DPAR_MIO_TOL_ABS_GAP", 1e-7)
# 定义变量
@variable(model, delta)                           # 标量 δ
# exponential variables
@variable(model, psi_1[1:N])                   
@variable(model, psi_2[1:N])                   
@variable(model, psi_3[1:N])        

@variable(model, phi_1[1:N])                   
@variable(model, phi_2[1:N])                   
@variable(model, phi_3[1:N]) 

@constraint(model, -L_arr' * psi_3 + sum(psi_2) + sum(phi_1) <= -delta)

for n in 1:N
    @constraint(model, [phi_3[n], phi_2[n], phi_1[n]] in MOI.DualExponentialCone())
end

for n in 1:N
    @constraint(model, phi_2[n] - phi_3[n] * M_arr[n] + psi_1[n] <= price[n] - delta)
end

for n in 1:N
    @constraint(model, [psi_3[n], psi_2[n], psi_1[n]] in MOI.DualExponentialCone())
end

@objective(model, Max, delta)

optimize!(model)
obj_val = objective_value(model);

In [ ]:
println("Objective value when X_RO = X_ETO:",obj_val)

- 我们再尝试求解存在两个指数cone的的对偶模型
$$
\begin{array}{clll}
     \min & \sum\limits_{n \in [N]} p_n \rho_n \\
     \mathrm{s.t.} & \rho_0 + \sum\limits_{n \in [N]} \rho_n = 1 \\
     & (\rho_n, \rho_0, - \rho_0 L_n) \in \mathcal{K}_{\exp} & \forall n \in [N] \\
      & (\rho_0, \rho_n, - \rho_n M_n ) \in \mathcal{K}_{\exp} & \forall n \in [N] \\
\end{array}
$$
<font color="red">
- 两个指数cone的的对偶模型，采用ETO的价格，得到的目标函数值等于ETO模型的目标函数值
</font>

In [ ]:
model = Model(Mosek.Optimizer)
# set_attribute(model, "mioTolRelGap", 0.00000001)
set_attribute(model, "MSK_DPAR_MIO_TOL_REL_GAP", 1e-7)
set_attribute(model, "MSK_DPAR_MIO_TOL_ABS_GAP", 1e-7)
# 定义变量
# exponential variables
@variable(model, rho_0)                   
@variable(model, rho[1:N])                   

@constraint(model, rho_0 >= 0)
@constraint(model, rho >= 0)
@constraint(model, rho_0 + sum(rho) == 1.0)
for n in 1:N
    @constraint(model, [-rho_0*L_arr[n], rho_0,rho[n]] in MOI.ExponentialCone())
end
for n in 1:N
    @constraint(model, [-rho[n]*M_arr[n],rho[n],rho_0] in MOI.ExponentialCone())
end
@objective(model, Min, price' * rho)
optimize!(model)
obj_val = objective_value(model)

In [ ]:
println("Objective value when X_RO = X_ETO:",obj_val)

### 我们尝试用双边指数锥

In [ ]:
P_dag

In [ ]:
p_ub = vec(maximum(P_dag,dims=2))
p_lb = vec(minimum(P_dag,dims=2))

In [ ]:
# phi_lb = p_lb .- p_ub

In [ ]:
phi_lb

In [ ]:
u = Obs_Feat
p_dag = P_dag

model = Model(Mosek.Optimizer)
# set_attribute(model, "QUIET", true)
# 定义变量
@variable(model, delta)                           # 标量 δ
@variable(model, omega_lb[1:N,1:N_u] >= 0)            
@variable(model, omega_ub[1:N,1:N_u] >= 0)           
@variable(model, pi_lb[1:N,1:N] >= 0)     
@variable(model, pi_ub[1:N,1:N] >= 0)     
@variable(model, eta_lb[1:N,1:N_u] >= 0)            
@variable(model, eta_ub[1:N,1:N_u] >= 0)           
@variable(model, ups_lb[1:N,1:N] >= 0)     
@variable(model, ups_ub[1:N,1:N] >= 0) 
# exponential variables
@variable(model, psi_1[1:N])                   
@variable(model, psi_2[1:N])                   
@variable(model, psi_3[1:N])        
@variable(model, phi_1[1:N])                   
@variable(model, phi_2[1:N])                   
@variable(model, phi_3[1:N])         

@variable(model, X[1:N, 1:K], Bin)        # 二进制变量 x_{jk}
@variable(model, Y[1:N,1:N,1:K])    
@variable(model, Z[1:N,1:N,1:K])    

for n in 1:N
    @constraint(model, omega_lb[n,:] .- omega_ub[n,:] .+ psi_3[n] .* u .== 0)
end

for n in 1:N
    @constraint(model, pi_lb[n,:] .- pi_ub[n,:] .+ sum(Y[n,:,:] .* p_dag,dims=2) .== 0)
end

for n in 1:N
    @constraint(model, eta_lb[n,:] .- eta_ub[n,:] .- phi_3[n] .* u .== 0)
end

for n in 1:N
    @constraint(model, ups_lb[n,:] .- ups_ub[n,:] .- sum(Z[n,:,:] .* p_dag,dims=2) .== 0)
end

@constraint(model, sum(omega_ub .* A_ub) - sum(omega_lb .* A_lb) + sum(pi_ub .* B_ub) - sum(pi_lb .* B_lb) + delta + sum(psi_2) + sum(phi_1) <= 0)

for n in 1:N
    @constraint(model, eta_ub[n,:]' * A_ub[n,:] - eta_lb[n,:]' * A_lb[n,:] + ups_ub[n,:]' * B_ub[n,:] - ups_lb[n,:]' * B_lb[n,:] + delta + psi_1[n] + phi_2[n] - X[n,:]' * p_dag[n,:] <= 0)
end

for n in 1:N
    @constraint(model, [psi_3[n], psi_2[n], psi_1[n]] in MOI.DualExponentialCone())
end

for n in 1:N
    @constraint(model, [phi_3[n],phi_2[n],phi_1[n]] in MOI.DualExponentialCone())
end

for n in 1:N
    for j in 1:N
        for k in 1:K
            @constraint(model, Y[n,j,k] >= psi_lb[n] * X[j,k])
            @constraint(model, Y[n,j,k] <= psi_ub[n] * X[j,k])
            # @constraint(model, Y[n,j,k] >= psi_3[n] - psi_ub * (1 - X[j,k]))
            # @constraint(model, Y[n,j,k] <= psi_3[n] - psi_lb * (1 - X[j,k]))
        end
    end
end

for n in 1:N
    for j in 1:N
        for k in 1:K
            @constraint(model, Z[n,j,k] >= phi_lb[n] * X[j,k])
            @constraint(model, Z[n,j,k] <= phi_ub[n] * X[j,k])
            # @constraint(model, Z[n,j,k] >= phi_3[n] - phi_ub * (1 - X[j,k]))
            # @constraint(model, Z[n,j,k] <= phi_3[n] - phi_lb * (1 - X[j,k]))
        end
    end
end

@constraint(model, sum(X,dims=2) .== 1)

# @constraint(model, X .- X_ETO .== 0)  # 固定X的值
for n in 1:N
    for j in 1:N
        @constraint(model, sum(Y[n,j,:]) == psi_3[n])
        @constraint(model, sum(Z[n,j,:]) == phi_3[n])
    end
end

@objective(model, Max, delta)

optimize!(model)
status = JuMP.termination_status(model)
# println("status: ", status)
# solution_summary(model)
if status == MOI.OPTIMAL
    obj_val = objective_value(model)
    X_val = round.(value.(X))
    solve_time = JuMP.solve_time(model)
else
    obj_val = NaN
    X_val = ones(N,N) .* NaN
    solve_time = NaN
end

In [ ]:
println("Objective value:",obj_val)

In [ ]:
X_RO2 = X_val
price_RO2 = sum(X_RO2 .* P_dag,dims=2)

In [ ]:
price_ETO

In [ ]:
price_given

In [ ]:
u = Obs_Feat
p_dag = P_dag
price_given = vec(price_ETO)
model = Model(Mosek.Optimizer)
# set_attribute(model, "QUIET", true)
# 定义变量
@variable(model, delta)                           # 标量 δ
@variable(model, omega_lb[1:N,1:N_u] >= 0)            
@variable(model, omega_ub[1:N,1:N_u] >= 0)           
@variable(model, pi_lb[1:N,1:N] >= 0)     
@variable(model, pi_ub[1:N,1:N] >= 0)     
@variable(model, eta_lb[1:N,1:N_u] >= 0)            
@variable(model, eta_ub[1:N,1:N_u] >= 0)           
@variable(model, ups_lb[1:N,1:N] >= 0)     
@variable(model, ups_ub[1:N,1:N] >= 0) 
# exponential variables
@variable(model, psi_1[1:N])                   
@variable(model, psi_2[1:N])                   
@variable(model, psi_3[1:N])        
@variable(model, phi_1[1:N])                   
@variable(model, phi_2[1:N])                   
@variable(model, phi_3[1:N])         

for n in 1:N
    @constraint(model, omega_lb[n,:] .- omega_ub[n,:] .+ psi_3[n] .* u .== 0)
end

for n in 1:N
    @constraint(model, pi_lb[n,:] .- pi_ub[n,:] .+ psi_3[n] .* price_given .== 0)
end

for n in 1:N
    @constraint(model, eta_lb[n,:] .- eta_ub[n,:] .- phi_3[n] .* u .== 0)
end

for n in 1:N
    @constraint(model, ups_lb[n,:] .- ups_ub[n,:] .- phi_3[n] .* price_given .== 0)
end

@constraint(model, sum(omega_ub .* A_ub) - sum(omega_lb .* A_lb) + sum(pi_ub .* B_ub) - sum(pi_lb .* B_lb) + delta + sum(psi_2) + sum(phi_1) <= 0)

for n in 1:N
    @constraint(model, eta_ub[n,:]' * A_ub[n,:] - eta_lb[n,:]' * A_lb[n,:] + ups_ub[n,:]' * B_ub[n,:] - ups_lb[n,:]' * B_lb[n,:] + delta + psi_1[n] + phi_2[n] - price_given[n] <= 0)
end

for n in 1:N
    @constraint(model, [psi_3[n], psi_2[n], psi_1[n]] in MOI.DualExponentialCone())
end

for n in 1:N
    @constraint(model, [phi_3[n],phi_2[n],phi_1[n]] in MOI.DualExponentialCone())
end

@objective(model, Max, delta)

optimize!(model)
status = JuMP.termination_status(model)
# println("status: ", status)
# solution_summary(model)
if status == MOI.OPTIMAL
    obj_val = objective_value(model)
    X_val = round.(value.(X))
    solve_time = JuMP.solve_time(model)
else
    obj_val = NaN
    X_val = ones(N,N) .* NaN
    solve_time = NaN
end

In [ ]:
obj_val